In [5]:

import torch.onnx
help(torch.onnx.export)

Help on function export in module torch.onnx:

export(model, args, f, export_params=True, verbose=False, training=False, input_names=None, output_names=None, aten=False, export_raw_ir=False, operator_export_type=None, opset_version=None, _retain_param_name=True, do_constant_folding=False, example_outputs=None, strip_doc_string=True, dynamic_axes=None)
    Export a model into ONNX format.  This exporter runs your model
    once in order to get a trace of its execution to be exported;
    at the moment, it supports a limited set of dynamic models (e.g., RNNs.)
    See also: :ref:`onnx-export`
    Arguments:
        model (torch.nn.Module): the model to be exported.
        args (tuple of arguments): the inputs to
            the model, e.g., such that ``model(*args)`` is a valid
            invocation of the model.  Any non-Tensor arguments will
            be hard-coded into the exported model; any Tensor arguments
            will become inputs of the exported model, in the order they


In [6]:
import torch.onnx
import torchvision

In [7]:
import timm

m = timm.create_model('mobilenetv3_100', pretrained=True)
print(m.eval())

GenEfficientNet(
  (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (conv_dw): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=Fa

In [8]:
dummy_input = torch.randn(1, 3, 224, 224)
# Obtain your model, it can be also constructed in your script explicitly
# model = torchvision.models.alexnet(pretrained=True)
# Invoke export
torch.onnx.export(m, dummy_input, "mobile.onnx")


In [9]:
import onnx

# Load the ONNX model
model = onnx.load("mobile.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %input.1[FLOAT, 1x3x224x224]
) initializers (
  %conv_stem.weight[FLOAT, 16x3x3x3]
  %bn1.weight[FLOAT, 16]
  %bn1.bias[FLOAT, 16]
  %bn1.running_mean[FLOAT, 16]
  %bn1.running_var[FLOAT, 16]
  %bn1.num_batches_tracked[INT64, scalar]
  %blocks.0.0.conv_dw.weight[FLOAT, 16x1x3x3]
  %blocks.0.0.bn1.weight[FLOAT, 16]
  %blocks.0.0.bn1.bias[FLOAT, 16]
  %blocks.0.0.bn1.running_mean[FLOAT, 16]
  %blocks.0.0.bn1.running_var[FLOAT, 16]
  %blocks.0.0.bn1.num_batches_tracked[INT64, scalar]
  %blocks.0.0.conv_pw.weight[FLOAT, 16x16x1x1]
  %blocks.0.0.bn2.weight[FLOAT, 16]
  %blocks.0.0.bn2.bias[FLOAT, 16]
  %blocks.0.0.bn2.running_mean[FLOAT, 16]
  %blocks.0.0.bn2.running_var[FLOAT, 16]
  %blocks.0.0.bn2.num_batches_tracked[INT64, scalar]
  %blocks.1.0.conv_pw.weight[FLOAT, 64x16x1x1]
  %blocks.1.0.bn1.weight[FLOAT, 64]
  %blocks.1.0.bn1.bias[FLOAT, 64]
  %blocks.1.0.bn1.running_mean[FLOAT, 64]
  %blocks.1.0.bn1.running_var[FLOAT, 64]
  %blocks.1.0.bn1.num_batches_trac

In [10]:
import onnxruntime as rt

sess = rt.InferenceSession('mobile.onnx')

print("The model expects input shape:", sess.get_inputs()[0].shape)
# print("image shape:", ximg.shape)

The model expects input shape: [1, 3, 224, 224]


## from pytorch2keras import pytorch_to_keras
import numpy as np
from torch.autograd import Variable
# we should specify shape of the input tensor
# k_model = pytorch_to_keras(m,input_shapes=dummy_input, verbose=True)  
input_np = np.random.uniform(0, 1, (1,3, 224, 224))
input_var = Variable(torch.FloatTensor(input_np))
k_model = pytorch_to_keras(m, input_var, [(3,224,224)], verbose=True)